In [1]:
from time import time
import numpy as np
import pandas as pd
import re
import nltk
from gensim.models import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [2]:
def load_data(filepath):
    """ Загружаем данные из CSV-файла в pandas DataFrame """
    df = pd.read_csv(filepath, delimiter=',', header=0, names=['sentence', 'label'])
    X = df['sentence']  # Извлекаем столбец с предложениями
    y = df['label']  # Извлекаем столбец с метками классов

    # Выводим информацию о данных
    print(f"Total sentences: {df.shape[0]}")
    print(f"Toxic: {df[df['label'] > 0]['label'].count()}")
    print(f"Not toxic: {df[df['label'] <= 0]['label'].count()}")
    return X, y

In [3]:
# Проверяем, есть ли отступы здесь
def preprocess_text(raw_text):
    """ Предварительная обработка текста """
    text = re.sub("[^0-9а-яА-Я]", " ", raw_text).lower()
    words = text.split()
    return words

In [4]:
# Эмбеддинги слов

def build_word2vec_model(sentences, num_features=300, min_word_count=40, context=20, downsampling=1e-3):
    # Обучаем модель Word2Vec на предложениях для получения эмбеддингов слов
    model = Word2Vec(sentences, vector_size=num_features, min_count=min_word_count, window=context, sample=downsampling)   # Инициализируем модель Word2Vec
    model.wv.init_sims(replace=True)  # Оптимизация модели для более быстрой работы
    return model


In [5]:
# Векторизация предложений

In [6]:
def make_feature_vec(words, model, num_features):
    # Создаем вектор признаков для предложения путем усреднения эмбеддингов слов
    feature_vec = np.zeros((num_features,), dtype="float32")  # Инициализируем пустой вектор признаков
    nwords = sum(1 for word in words if word in model.wv.index_to_key)  # Количество слов, у которых есть эмбеддинг

    if nwords == 0:
        return feature_vec

    for word in words:
        if word in model.wv.index_to_key:
            # Добавляем эмбеддинг слова к вектору признаков
            feature_vec = np.add(feature_vec, model.wv[word])

    # Усредняем вектор признаков по количеству слов с эмбеддингом
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec

In [7]:
def get_avg_feature_vecs(sentences, model, num_features):
    # Создаем матрицу векторов признаков для всех предложений
    review_feature_vecs = np.zeros((len(sentences), num_features), dtype="float32")  # Инициализируем пустую матрицу векторов признаков

    for idx, sentence in enumerate(sentences):
        # Создаем вектор признаков для каждого предложения
        review_feature_vecs[idx] = make_feature_vec(sentence, model, num_features)

    return review_feature_vecs

In [8]:
# Модель классификатора

In [9]:
def train_classifier(X_train, y_train, batch_size=200, total_epochs=10, layers=(300,40,30)):
    # Обучаем классификатор на векторах признаков предложений
    clf = MLPClassifier(solver='adam', hidden_layer_sizes=layers, random_state=1)
    start_time = time()

    for epoch in range(total_epochs):
        for i in range(0, len(X_train), batch_size):
            # Обучаем классификатор по батчам данных
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            clf.partial_fit(X_batch, y_batch, classes=np.unique(y_train))

        if time() - start_time > total_epochs * 60:  # Ограничение времени выполнения в минутах
            break

    return clf

In [10]:
# Основной код
if __name__ == "__main__":
    # Загружаем данные
    X, y = load_data("./data/labeled_rutoxic.csv")

    # Предобработка и разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    sentences_train = X_train.apply(preprocess_text)
    sentences_test = X_test.apply(preprocess_text)

# Построение модели Word2Vec
word2vec_model = build_word2vec_model(sentences_train)

# Векторизация предложений
train_vectors = get_avg_feature_vecs(sentences_train, word2vec_model, 300)
test_vectors = get_avg_feature_vecs(sentences_test, word2vec_model, 300)

# Обучение классификатора
clf = train_classifier(train_vectors, y_train.values)

# Оценка классификатора
predicted = clf.predict(test_vectors)
accuracy = np.mean(predicted == y_test.values)
print(f"Accuracy: {accuracy:.2%}")
# Фильтрация и сохранение токсичных комментариев
toxic_comments = X_test[predicted == 1]  # Меняем здесь на 1, если токсичность кодируется единицей
toxic_df = pd.DataFrame(toxic_comments)
toxic_df['label'] = 1  # предполагаем, что этикетка токсичных комментариев - это 1

# Сохраняем токсичные комментарии в CSV-файл
toxic_df.to_csv("toxic_comments.csv", index=False)

Total sentences: 14412
Toxic: 4826
Not toxic: 9586


/tmp/ipykernel_11141/817837877.py:6: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.wv.init_sims(replace=True)  # Оптимизация модели для более быстрой работы


Accuracy: 75.12%


In [11]:
 # Создаем DataFrame с результатами
results = pd.DataFrame({"Actual": y_test, "Predicted": y_pred})

    # Сохраняем результаты в CSV-файл
results.to_csv("results.csv", index=False)

NameError: name 'y_pred' is not defined